## SFT train the d20 model

I'm going to approach this like `challenge-28-midtrain-d20/midtrain-d20.ipynb`. I'll use the 8xH100 in lambda cloud and run all the scripts from this notebook.

Follow the instructions here to get the machine ready: `challenge-28-midtrain-d20/midtrain-d20.ipynb`

In [2]:
!nvidia-smi

Fri Nov 21 00:31:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:61:00.0 Off |                    0 |
| N/A   33C    P0             71W /  700W |       0MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

#### Repeat CORE eval on base model

For reasons discussed in `challenge-28-midtrain-d20/investigate-questions.ipynb`, I want to first do the CORE eval on the base model **with max-per-task=500**

In [3]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_base_eval -- \
--source=base --model-tag=d20 --max-per-task=500

W1121 00:32:12.588000 14480 torch/distributed/run.py:803] 
W1121 00:32:12.588000 14480 torch/distributed/run.py:803] *****************************************
W1121 00:32:12.588000 14480 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 00:32:12.588000 14480 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.

#### Redo mid training

Because I left out updating muon momentum (noticed in `challenge-29-understand-sft/understand-sft.ipynb`), I want to redo the mid training.

In [4]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_mid_train -- \
--model_tag=d20 --run=challenge-30-1

W1121 00:33:11.705000 39756 torch/distributed/run.py:803] 
W1121 00:33:11.705000 39756 torch/distributed/run.py:803] *****************************************
W1121 00:33:11.705000 39756 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 00:33:11.705000 39756 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

#### Do chat eval on mid model

In [5]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=mid --model-tag=d20

W1121 00:43:10.581000 49333 torch/distributed/run.py:803] 
W1121 00:43:10.581000 49333 torch/distributed/run.py:803] *****************************************
W1121 00:43:10.581000 49333 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 00:43:10.581000 49333 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

#### SFT train

First do a few iterations to make sure everything works.

In [8]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --num_iterations=10 --run=challenge-30-2

W1121 00:55:22.014000 53214 torch/distributed/run.py:803] 
W1121 00:55:22.014000 53214 torch/distributed/run.py:803] *****************************************
W1121 00:55:22.014000 53214 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 00:55:22.014000 53214 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

Then do for real:

In [9]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_sft -- \
--model_tag=d20 --run=challenge-30-3

W1121 00:56:40.397000 54574 torch/distributed/run.py:803] 
W1121 00:56:40.397000 54574 torch/distributed/run.py:803] *****************************************
W1121 00:56:40.397000 54574 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 00:56:40.397000 54574 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

^ Did not seem like it trained properly from the loss jumping all over the place. Number of tokens per step also seems very low.

#### Do chat eval on sft model

In [10]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=sft --model-tag=d20

W1121 01:10:47.798000 57235 torch/distributed/run.py:803] 
W1121 01:10:47.798000 57235 torch/distributed/run.py:803] *****************************************
W1121 01:10:47.798000 57235 torch/distributed/run.py:803] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1121 01:10:47.798000 57235 torch/distributed/run.py:803] *****************************************
Autodetected device type: cuda
/home/ubuntu/learn-nanochat/.venv/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Py

(while chat eval is running, remember to sftp the mid and sft checkpoints to my mac)

### Back on my mac

Terminated the lambda cloud machine. Used it for 0.88 hr at a cost of $21.02.

There are a lot of things to look into. I'm also thinking I should add the reporting stuff soon because it's painful trying to compare accuracy numbers, etc. by looking back at output.

Observations:

- The CORE metric eval on the base model with max-per-task=500 came out to 0.2084. This was expected and confirms the answer to question 2 in `challenge-28-midtrain-d20/investigate-questions.ipynb`.

- Final validation bpb on the retrained mid model was 0.3961. This is worse than the 0.3947 from previous training in `challenge-28-midtrain-d20/midtrain-d20.ipynb`.

- Here's what the chat eval was on previous mid from `challenge-28-midtrain-d20/midtrain-d20.ipynb`) and retrained mid and sft trained from this notebook:

```
                  previous mid     retrained mid        sft

ARC-Easy          42.80            43.18                44.07
ARC-Challenge     31.31            33.19                31.74
MMLU              32.53            33.07                32.34
GSM8K             2.81             2.20                 3.71
HumanEval         9.76             6.71                 6.10
SpellingBee       96.88*           97.27                95.70

* different test set due to fixing train/test split bug
```

Observations on the SFT train:

- num_tokens per step seems very small compared to the ~500,000 we were seeing before. Not sure it's wrong but should git into it.
 
- Training loss jumps around a lot and doesn't seem to decrease or increase overall. Again, not sure it's wrong, but look into it.

- Validation loss is 1.011 at step 0 then jumps to 1.015 at step 100 and declines to 1.04 at the end. Is this expected?

- Accuracy of ARC-Easy, which is part of the sft training data, goes up compared to mid. But the accuracy of ARC-Challenge, also part of the sft training data, goes down.

- Accuracy of MMLU which is not part of the sft training data goes down.

- Accuracy of GSM8K which is part of the sft training data goes up.

- Accuracy of HumanEval, which is not part of the sft training data, goes down. (Reminder to look at examples that are right for HumanEval. I'm surprised it can even 6% right.)

- Accuracy of SpellingBee, which is part of the sft training data (but a very small part especially compared to what a big percent it is in mid training), went down.

Also, look into the warning: UserWarning: Please use the new API settings to control TF32 behavior

In shor there's a lot to look into!